# Variables are stored in both Rows and Columns

## "Housekeeping"

In [1]:
% load_ext watermark
% watermark -d -t -v -z -p numpy,pandas

2018-08-26 12:56:31 CEST

CPython 3.6.5
IPython 6.5.0

numpy 1.15.1
pandas 0.23.4


In [2]:
import numpy as np
import pandas as pd

In [3]:
pd.set_option('display.max_columns', 40)

## Example: Weather

The [Global Historical Climatology Network](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn) collects daily weather. For this example, data for one weather station (MX17004) in Mexico are used.

### Load the Data

The raw dataset comes in a format that is a mixture of a fixed-width style with occasional usage of characters as seperators. Some tedious cleaning work is necessary.

In [4]:
# Extract the data as one column and
# use string slicing to obtain groups of columns.
weather = pd.read_csv('data/weather.txt', header=None, sep='^')

# First, remove the weird character seperators,
# then split the columns by whitespace, and
# finally name them appropriately.
days = (
    weather[0]
    .map(lambda x: x[21:]).str.replace('OI', '  ')
    .str.replace('OS', '  ').str.replace('SI', '  ').str.replace('I', ' ')
    .str.replace('S', ' ').str.replace('B', ' ').str.replace('D', ' ')
    .map(str.lstrip).str.split(r'\s+', expand=True)
)[list(range(31))].rename(columns={i: f'd{i+1}' for i in range(31)})

# The non-temperature columns can be extracted as simple slices.
weather = pd.DataFrame(data={
    'id': weather[0].map(lambda x: x[:11]),
    'year': weather[0].map(lambda x: x[11:15]).astype(int),
    'month': weather[0].map(lambda x: x[15:17]).astype(int),
    'element': weather[0].map(lambda x: x[17:21]).str.lower(),
})

# The temperatures were stored as whole integers
# with -9999 indicating missing values.
for i in range(1, 32):
    weather[f'd{i}'] = days[f'd{i}'].astype(float) / 10
weather = weather.replace(-999.9, np.NaN)

# Discard the non-temperature observations and
# sort the dataset as in the paper.
weather = (
    weather[weather['element'].isin(['tmax', 'tmin'])]
    .sort_values(['id', 'year', 'month', 'element'])
    .reset_index(drop=True)
)

### Messy Data

Below is a dataset assumed to have been provided like this as "raw", i.e., the data analyst did not do the above parsing work but some third party instead.

> The most complicated form of messy data occurs when variables are stored in both rows and columns. Table 11 shows daily weather data from the Global Historical Climatology Network for one weather station (MX17004) in Mexico for five months in 2010. It has variables in
individual columns (*id*, *year*, *month*), spread across columns (day, d1–d31) and across rows (*tmin*, *tmax*) (minimum and maximum temperature). Months with less than 31 days have
structural missing values for the last day(s) of the month. The *element* column is not a variable; it stores the names of variables.

In [5]:
weather[(weather['year'] == 2010)].head(10)

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
1099,MX000017004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1100,MX000017004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
1101,MX000017004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1102,MX000017004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103,MX000017004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN,NaN,34.5,NaN,NaN,NaN,NaN,NaN,31.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1104,MX000017004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN,NaN,16.8,NaN,NaN,NaN,NaN,NaN,17.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,MX000017004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.3,NaN,NaN,NaN,NaN
1106,MX000017004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN
1107,MX000017004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.2,NaN,NaN,NaN,NaN
1108,MX000017004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.2,NaN,NaN,NaN,NaN


### Molten Data

> To tidy this dataset we first melt it with colvars *id*, *year*, *month* and the column that contains variable names, *element* [...]. For presentation, we have dropped the missing values, making them implicit rather than explicit. This is permissible because we know how many days are in each month and can easily reconstruct the explicit missing values.

In [6]:
# Melt the dataset and extract a date column.
molten_weather = (
    pd.melt(weather, id_vars=['id', 'year', 'month', 'element'], var_name='day')
    .assign(day=lambda x: x['day'].str.extract('(\d+)').astype(int))
    .assign(date=lambda x: pd.to_datetime(x[['year', 'month', 'day']], errors='coerce'))
)[['id', 'date', 'element', 'value']]

# Make the missing values implicit.
molten_weather = molten_weather[molten_weather['value'].notnull()]

# Sort the data as in the paper.
molten_weather = (
    molten_weather
    .sort_values(['id', 'date', 'element'])
    .reset_index(drop=True)
)

> This dataset is mostly tidy, but we have two variables stored in rows: *tmin* and *tmax*, the type of observation.

In [7]:
molten_weather[(molten_weather['date'].dt.year == 2010)].head(10)

,id,date,element,value
23183,MX000017004,2010-01-30,tmax,27.8
23184,MX000017004,2010-01-30,tmin,14.5
23185,MX000017004,2010-02-02,tmax,27.3
23186,MX000017004,2010-02-02,tmin,14.4
23187,MX000017004,2010-02-03,tmax,24.1
23188,MX000017004,2010-02-03,tmin,14.4
23189,MX000017004,2010-02-11,tmax,29.7
23190,MX000017004,2010-02-11,tmin,13.4
23191,MX000017004,2010-02-23,tmax,29.9
23192,MX000017004,2010-02-23,tmin,10.7


### Tidy Data

> Fixing this requires the cast, or unstack, operation. This performs the inverse of melting by rotating the element variable back out into the columns

Note that [pd.DataFrame.unstack](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html) method uses a DataFrame's index as columns to unstack over.

In [8]:
tidy_weather = molten_weather.set_index(['id', 'date', 'element']).unstack()

# Make the column headers look as in the paper.
tidy_weather.columns = tidy_weather.columns.droplevel(0)
tidy_weather.columns.name = None
tidy_weather = tidy_weather.reset_index()

> This form is tidy. There is one variable in each column, and each row represents a day’s observations.

In [9]:
tidy_weather[(tidy_weather['date'].dt.year == 2010)].head(10)

,id,date,tmax,tmin
12087,MX000017004,2010-01-30,27.8,14.5
12088,MX000017004,2010-02-02,27.3,14.4
12089,MX000017004,2010-02-03,24.1,14.4
12090,MX000017004,2010-02-11,29.7,13.4
12091,MX000017004,2010-02-23,29.9,10.7
12092,MX000017004,2010-03-05,32.1,14.2
12093,MX000017004,2010-03-10,34.5,16.8
12094,MX000017004,2010-03-16,31.1,17.6
12095,MX000017004,2010-04-27,36.3,16.7
12096,MX000017004,2010-05-27,33.2,18.2
